In [37]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

Xtrain = np.array(pd.read_csv('csv/x_train_significant.csv', sep=','))
ytrain = np.array(pd.read_csv('csv/y_train.csv', sep=',', header=None))
Xtest = np.array(pd.read_csv('csv/x_test_significant.csv', sep=','))
ytest = np.array(pd.read_csv('csv/y_test.csv', sep=',', header=None))

In [38]:
sample_leaf_options = [1,5,10,50,100,200,500]
max_score = -1
max_score_leaf = 0
fewest_bad_pairs = ytest.shape[0]/2
fewest_bad_pairs_leaf = 0
for min_leaf in sample_leaf_options:
    clf = RandomForestClassifier(n_estimators=200, max_features="sqrt", min_samples_leaf=min_leaf, oob_score = True)
    clf = clf.fit(Xtrain, np.ravel(ytrain))

    p = clf.predict(Xtest)
    score = clf.score(Xtest, ytest, sample_weight=None)
    if (score > max_score):
        max_score = score
        max_score_leaf = min_leaf

    bad_pair = 0
    for i in range(int(p.shape[0]/2)):
        if p[2*i]==p[2*i+1]:
            bad_pair+=1
    if (bad_pair < fewest_bad_pairs):
        fewest_bad_pairs = bad_pair
        fewest_bad_pairs_leaf = min_leaf
print('Best rate:', max_score, max_score_leaf, 'min_leaf')
print('Fewest bad pairs:', fewest_bad_pairs, fewest_bad_pairs_leaf, 'min_leaf')

('Best rate:', 0.64715447154471539, 5, 'min_leaf')
('Fewest bad pairs:', 41, 100, 'min_leaf')


In [39]:
n_estimators = [50,100,200,500,750,1000]
max_score = -1
max_score_trees = 0
fewest_bad_pairs = p.shape[0]/2
fewest_bad_pairs_trees = 0
for n_trees in n_estimators:
    clf = RandomForestClassifier(n_estimators=n_trees, max_features="sqrt", 
                                 min_samples_leaf=max_score_leaf, oob_score = True)
    clf = clf.fit(Xtrain, np.ravel(ytrain))

    p = clf.predict(Xtest)
    score = clf.score(Xtest, ytest, sample_weight=None)
    if (score > max_score):
        max_score = score
        max_score_trees = min_leaf

    bad_pair = 0
    for i in range(int(p.shape[0]/2)):
        if p[2*i]==p[2*i+1]:
            bad_pair+=1
    if (bad_pair < fewest_bad_pairs):
        fewest_bad_pairs = bad_pair
        fewest_bad_pairs_trees = min_leaf
print('Best rate:', max_score, max_score_trees, 'trees')
print('Fewest bad pairs:', fewest_bad_pairs, fewest_bad_pairs_trees, 'trees')

('Best rate:', 0.65000000000000002, 500, 'trees')
('Fewest bad pairs:', 38, 500, 'trees')


In [40]:
clf = RandomForestClassifier(n_estimators=max_score_trees, max_features="sqrt", 
                                 min_samples_leaf=max_score_leaf, oob_score = True)
clf = clf.fit(Xtrain, np.ravel(ytrain))
p = clf.predict(Xtest)
probs = clf.predict_proba(Xtest)
clf.score(Xtest, ytest)

0.64634146341463417

In [26]:
games_test = pd.read_csv('games_test.csv', header=None)
wins = {}
for i in range(len(games_test)):
    away_win = (probs[2*i][1]+probs[2*i+1][0])/2
    home_win = (probs[2*i][0]+probs[2*i+1][1])/2
    away = games_test.loc[i,0]
    home = games_test.loc[i,2]
    if away not in wins:
        wins[away] = away_win
    else:
        wins[away] += away_win
    if home not in wins:
        wins[home] = home_win
    else:
        wins[home] += home_win

In [31]:
import pred_to_record
pred_to_record.to_csv(pred_to_record.pred_to_record(p), 'random_forest')
pred_to_record.to_csv(pred_to_record.pred_to_record([prob[1] for prob in probs]), 'random_forest_prob')

10.25508004194344

In [34]:
print(pred_to_record.rms(p))
print(pred_to_record.rms([prob[1] for prob in probs]))

17.28390388


10.229010715221683